# VacationPy

In [1]:
# declarations
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
from api_keys import g_key
import gmaps
import os
import time
import scipy.stats as st
from datetime import datetime
import json

In [2]:
# initializations
citiesfilepath = "..\WeatherPy\output_data\cities.csv"

gmaps.configure(api_key = g_key)
base_url = "https://maps.googleapis.com"
end_point = base_url + "/maps/api/place/nearbysearch/json?"
params = {"location": "", "radius": 5000, "type": "lodging", "key": g_key}

### Store Part I results into DataFrame

In [3]:
citiesDF = pd.read_csv(citiesfilepath)
citiesDF = citiesDF[["City","Lat","Lng","Max Temp","Humidity","Cloudiness","Wind Speed","Country","Date"]]

In [4]:
citiesDF.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,saint-philippe,-21.3585,55.7679,76.59,73,18,10.83,RE,1658827666
1,butaritari,3.0707,172.7902,82.20,73,99,16.82,KI,1658827667
2,hilo,19.7297,-155.0900,81.28,92,75,4.61,US,1658827667
3,kaitangata,-46.2817,169.8464,45.14,94,100,16.06,NZ,1658827668
4,narsaq,60.9167,-46.0500,48.42,82,88,1.30,GL,1658827668


### Humidity Heatmap

In [5]:
locations = citiesDF[["Lat", "Lng"]].astype(float)
humidity = citiesDF["Humidity"].astype(float)

# define heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius = 1)

In [6]:
# create figure
fig = gmaps.figure()
fig.add_layer(heat_layer)

# show figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria

In [7]:
citiesDF.dropna(inplace = True)

citiesDF = citiesDF[(citiesDF["Max Temp"] > 70) 
                    & (citiesDF["Max Temp"] < 80) 
                    & (citiesDF["Wind Speed"] < 10) 
                    & (citiesDF["Cloudiness"] == 0)]

### Hotel Map

In [8]:
hotel_df = citiesDF.copy()
hotel_df["Hotel Name"] = ""
hotel_df["City"] = hotel_df["City"].str.title()

In [9]:
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
63,Ascension,30.2169,-90.9001,79.05,96,0,0.00,US,1658827734,
189,Riverton,40.5219,-111.9391,79.84,43,0,8.01,US,1658827801,
215,Russell,32.3502,-85.1999,74.80,94,0,3.44,US,1658827881,
290,Bethel,41.3712,-73.4140,70.16,76,0,4.05,US,1658827995,
367,Istisu,40.9457,48.0738,77.67,47,0,8.70,AZ,1658828067,


In [10]:
for index, row in hotel_df.iterrows():     
    # set param
    params['location'] = f"{row['Lat']},{row['Lng']}"  
    
    # api call
    response = requests.get(end_point, params=params).json()  
    
    # update Hotel Name in dataframe
    try:
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]["name"]
    except IndexError:
        hotel_df.loc[index, 'Hotel Name'] = "No lodging"

In [11]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
63,Ascension,30.2169,-90.9001,79.05,96,0,0.00,US,1658827734,Highland Inn
189,Riverton,40.5219,-111.9391,79.84,43,0,8.01,US,1658827801,Ramada by Wyndham Draper
215,Russell,32.3502,-85.1999,74.80,94,0,3.44,US,1658827881,Macs
290,Bethel,41.3712,-73.4140,70.16,76,0,4.05,US,1658827995,Courtyard by Marriott Danbury
367,Istisu,40.9457,48.0738,77.67,47,0,8.70,AZ,1658828067,Shirvan Resort (Halal)
428,Jackson,42.4165,-122.8345,75.07,56,0,0.00,US,1658828134,Agate Village Apartments
434,Portland,45.5234,-122.6762,74.16,70,0,4.61,US,1658828137,Embassy Suites by Hilton Portland Downtown
450,Saint George,37.1041,-113.5841,72.19,83,0,0.00,US,1658828156,Best Western Plus Abbey Inn
454,Sikeston,36.8767,-89.5879,76.91,100,0,0.00,US,1658828188,Drury Inn & Suites Sikeston
468,Talladega,33.4359,-86.1058,73.51,94,0,0.00,US,1658828173,"Holiday Inn Express & Suites Talladega, an IHG..."


In [12]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]].copy()

In [13]:
# define hotel marker layer
hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# add marker layer to original figure
fig.add_layer(hotel_layer)

# display figure
fig

Figure(layout=FigureLayout(height='420px'))